In [46]:
# FUNCTION DECLARATIONS

# getAspectDescription(text: string) => [{aspect: string, description: string}]
# getSentiment(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

# getAspectDescription

In [47]:
# %pip install spacy

In [48]:
# We get started by importing spacy
import spacy
nlp = spacy.load("en_core_web_lg")

In [49]:
# getAspectDescription(text: string) => [{aspect: string, description: string}]

def getAspectDescription(text):
    """
    This function takes in a string of text and returns a list of dictionaries, each containing the aspect and its corresponding description.

    Args:
        text (str): The input text to analyze.

    Returns:
        list: A list of dictionaries, each containing the aspect and its corresponding description.

    Example:
        >>> text = "The camera is great but the battery life is short."
        >>> getAspectDescription(text)
        [{'aspect': ['camera'], 'description': 'great'}, {'aspect': ['battery', 'life'], 'description': 'short'}]
    """
    aspects = []

    doc = nlp(text)
    descriptive_term = ''
    target = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            # target = token.text
            target.append(token.text)
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = f"{prepend}" + token.text

    aspects.append({'aspect': target, 'description': descriptive_term})

    return aspects


In [50]:
getAspectDescription('The product is strong, but it was not the right color. Poor packaging')

[{'aspect': ['product'], 'description': 'Poor'}]

# getSentiment

In [51]:
# %pip install pandas
# %pip install re
# %pip install bs4
# %pip install nltk
# %pip install pickle

In [52]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

In [53]:
# getSentiment(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

model = pickle.load(open('modelNB.pkl', 'rb'))

def getSentiment(raw_text: str):
    """
    This function takes in a string of raw text and performs sentiment analysis to determine whether the text is positive or negative. It returns a tuple consisting of the sentiment label and the probability of the prediction.

    Args:
        raw_text (str): The raw text to analyze.

    Returns:
        tuple: A tuple consisting of the sentiment label and the probability of the prediction.

    Example:
        >>> raw_text = "This product is amazing! I love it so much."
        >>> getSentiment(raw_text)
        ('Positive', array([[0.00819811, 0.99180189]]))
    """

    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()

    # Remove HTML
    review_text = BeautifulSoup(raw_text).get_text()

    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # Convert words to lower case and split each word up
    words = letters_only.lower().split()

    # Convert stopwords to a set
    stops = set(stopwords.words('english'))

    # Adding on stopwords that were appearing frequently in both positive and negative reviews
    stops.update(['app','shopee','shoppee','item','items','seller','sellers','bad'])

    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]

    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

    # Join words back into one string, with a space in between each word
    final_text = pd.Series(" ".join(meaningful_words))

    # Generate predictions
    pred = model.predict(final_text)[0]
    probability = model.predict_proba([pd.Series.to_string(final_text)])


    if pred == 1:
        output = "Negative"
    else:
        output = "Postive"

    return output, probability


In [54]:
text = 'The product is strong, but it was not the right color. Poor packaging'

In [55]:
output, proba = getSentiment(text)

In [56]:
print(f"Text: {text}", output, proba, sep='\t')

Text: The product is strong, but it was not the right color. Poor packaging	Negative	[[0.22424745 0.77575255]]


# Aspect Sentiment Classification

In [57]:
text = 'The food we had yesterday was delicious'
aspects = getAspectDescription(text)

aspectSentiment = [{'aspect': aspect['aspect'], 'description': aspect['description'], 'sentiment': getSentiment(aspect['description'])} for aspect in aspects]


print(aspectSentiment)

[{'aspect': ['food'], 'description': 'delicious', 'sentiment': ('Postive', array([[0.60999281, 0.39000719]]))}]


Craig

In [58]:
sentences = [
    "The phone has a huge display that is crystal clear. I love watching videos on it.",
    "This phone is compact and fits perfectly in my pocket. The size is just right.",
    "The camera on this phone is amazing. The pictures come out clear and vivid.",
    "The battery life on this phone is incredible. I can go all day without needing to charge it.",
    "I'm impressed with the phone's sleek design. It looks great and feels good in my hand.",
    "The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.",
    "The size of this phone is perfect for one-handed use. It's not too big or too small.",
    "The camera on this phone takes amazing low light photos. I'm really happy with the quality.",
    "The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.",
    "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight."
]

#keys = []

#for sentence in sentences:
    #aspects = getAspectDescription(sentence)
    #for aspect in aspects:
        #keys.append(str(aspect['aspect']))
#aspect_dict = {key: [] for key in keys}

#for item in aspectSentiments:
    #sentiment = item['sentiment']
    #for aspect in aspects:
        #if aspect not in aspect
    
#for key in keys:
    #print(key)


aspect_dict = {}
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    for aspect in aspects:
        aspect_name = tuple(aspect['aspect'])
        for item in aspectSentiment:
            sentiment = item['sentiment']
        if aspect_name not in aspect_dict:
            aspect_dict[aspect_name] = {}
        aspect_dict[aspect_name][sentence] = sentiment

for aspect, sentences in aspect_dict.items():
    print(f"{aspect}:")
    for sentence, sentiment in sentences.items():
        print(f" - {sentence}: {sentiment}")

('phone',):
 - The phone has a huge display that is crystal clear. I love watching videos on it.: ('Postive', array([[0.60999281, 0.39000719]]))
('phone', 'size'):
 - This phone is compact and fits perfectly in my pocket. The size is just right.: ('Postive', array([[0.60999281, 0.39000719]]))
('camera', 'pictures'):
 - The camera on this phone is amazing. The pictures come out clear and vivid.: ('Postive', array([[0.60999281, 0.39000719]]))
('life',):
 - The battery life on this phone is incredible. I can go all day without needing to charge it.: ('Postive', array([[0.60999281, 0.39000719]]))
():
 - I'm impressed with the phone's sleek design. It looks great and feels good in my hand.: ('Postive', array([[0.60999281, 0.39000719]]))
 - The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.: ('Postive', array([[0.60999281, 0.39000719]]))
 - I'm in love with the phone's display. It's so clear and bright, even in direct sunlight.: ('Postive', array(